In [ ]:
import pandas as pd
import numpy as np
import os 

### Read the input and output of the simulator

##### Define the path to the folder of the outputs and the samples.csv
The outputs here are generated using ModularCirc Library (Tutorial_03)

In [ ]:
out_path = '/Users/mfamili/work/ModularCirc/Tutorials/Tutorial_03/Outputs/Out_01'
in_path = '/Users/mfamili/work/ModularCirc/Tutorials/Tutorial_03/samples_Naghavi.csv'

In [ ]:
parameters = pd.read_csv(in_path)
len(parameters)

##### Read and process outputs
- Read the outputs and convert them to a list of dictionaries. report if any output files are missing
- convert outputs to a numpy array of (number of samples, number of outputs)

In [ ]:
outputs = []
num_sim = 1000
no_sim_result_list = []
# Loop through the expected range of files
for i in range(num_sim):
    filename = f"all_outputs_{i}.csv"
    file_path = os.path.join(out_path, filename)
    
    if os.path.exists(file_path):  # Check if the file exists
        try:
            # Read the DataFrame and append it to the list
            df = pd.read_csv(file_path)
            outputs.append(df)
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            outputs.append(False)  # Append False if there's an error
    else:
        no_sim_result_list.append(i)
        print(f"File {file_path} doesn't exist")
        outputs.append(False)  # Append False if the file doesn't exist

outs = np.array([df.max().values - df.min().values for df in outputs])
print(np.array(outs).shape)
print(np.array(outs).dtype)


##### Update the parameters to drop the parameter sets where the output is missing

In [ ]:
updated_parameters = parameters.drop(no_sim_result_list)
updated_parameters= updated_parameters.to_numpy()
updated_parameters.shape

In [ ]:
preprocessing_methods = [
    {"name": "PCA", "params": {"reduced_dim": 8}},
    {
        "name": "VAE",
        "params": {"reduced_dim": 3, "hidden_layers": [64, 32], "epochs": 100},
    },]
#    {"name": "None", "params": {}},]



#preprocessing_methods = [{"name": "PCA", "params": {"n_components": 8}}]

##### Setup autoemulate 
- here we are choosing to preprocess the outputs in order to reduce the dimentionality


In [ ]:
from autoemulate.compare import AutoEmulate
em = AutoEmulate()
em.setup(updated_parameters, outs, models=["gp"], preprocessing_methods =preprocessing_methods)


In [ ]:
best_model = em.compare()

In [ ]:
best_model

In [ ]:
em.preprocessing_results.keys()


In [ ]:
em.summarise_cv(model='gp')

In [ ]:
em.summarise_cv()  # Skip None models by filtering

In [ ]:
em.evaluate(em.get_model()) 

In [ ]:
em.plot_eval(em.get_model(), input_index=[0,1,2,3,4], output_index=[0,1,2])

In [ ]:
em.plot_cv(style="residual_vs_predicted")